<a href="https://colab.research.google.com/github/IvanPP1018/SIMULACION-1/blob/main/EXAMEN_SANTANDER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**EXAMEN SANTANDER**

**FECHA: 04/03/2025**

**AUTOR: IVAN PÁEZ PÉREZ**

**REPORTE: ESTATUS TARJETAS**

In [119]:
#Declaramos librerias
import pandas as pd
import numpy as np
from datetime import date

In [120]:
# Cargar archivos Excel
ESTATUS_CLIENTES = "ESTATUS CLIENTES.xlsx"
ESTATUS_CTAS = "ESTATUS CTAS.xlsx"
OPERACIONES = "OPERACIONES.xlsx"

In [121]:
# Leer los datos de los archivos Excel
datos_clientes = pd.read_excel(ESTATUS_CLIENTES)
datos_cuentas = pd.read_excel(ESTATUS_CTAS)
datos_operaciones = pd.read_excel(OPERACIONES)

In [122]:
# Convertir los DataFrames a listas de diccionarios para facilitar el acceso
lista_clientes = datos_clientes.to_dict(orient='records')
lista_cuentas = datos_cuentas.to_dict(orient='records')
lista_operaciones = datos_operaciones.to_dict(orient='records')

In [123]:
# Definir meses y sus días correspondientes
MESES = {10: 31, 11: 30, 12: 31}
Dat_fes = [{10:'OCTUBRE'},{11:'NOVIEMBRE'},{12:'DICIEMBRE'}]

In [124]:
# Función para obtener la fecha correspondiente al mes
def fecha(mes):
    """Retorna la fecha correspondiente al mes ingresado."""
    if mes in MESES:
        return date(2024, mes, MESES[mes])
    else:
        return "Mes no válido"

In [126]:
# Función para obtener las cuentas de un cliente
def obtener_cuentas_cliente(numero_cliente):
    """Retorna las cuentas asociadas a un cliente."""
    return datos_cuentas[datos_cuentas['NUM_CLIENTE'] == numero_cliente]['CUENTA'].unique()

In [127]:
"""Retorna una lista con el tipo de tarjeta (TDC o TDD) para cada cuenta."""
def Tarjetas_d_o_c(cuenta_cliente):
  debido_o_credito = []
  for kk in cuenta_cliente:
    for zz in range(len(lista_cuentas)):
      if kk == lista_cuentas[zz]['CUENTA']:
        if lista_cuentas[zz]['PRODUCTO'] == 'TDC':
          nombre = 'TDC'
        else:
          nombre = 'TDD'
    debido_o_credito.append(nombre)
    A12 = debido_o_credito
  return A12

In [128]:
"""Retorna una lista con el tipo de tarjeta (100 o 200) para cada tarjeta"""
def Resulta(Lista):
  resultado = [100 if x == 'TDC' else 200 for x in Lista]
  return resultado

In [129]:
# Función para calcular la edad del cliente
def calcular_edad(numero_cliente):
    """Calcula la edad del cliente basado en su RFC."""
    for cliente in lista_clientes:
        if cliente['NUM_CLIENTE'] == numero_cliente:
            rfc = cliente['RFC']
            año = int(rfc[4:6])
            mes_nacimiento = int(rfc[6:8])
            dia_nacimiento = int(rfc[8:10])
            año = 1900 + año if año > 25 else 2000 + año
            edad = fecha(mes).year - date(año, mes_nacimiento, dia_nacimiento).year
            return edad
    return None

In [130]:
# Función para determinar el nivel socioeconómico
def nivel_socioeconomico(numero_cliente):
    """Retorna el nivel socioeconómico basado en el ingreso mensual."""
    for cliente in lista_clientes:
        if cliente['NUM_CLIENTE'] == numero_cliente:
            ingreso = cliente['INGRESO_MENSUAL']
            if ingreso >= 85000:
                return 'A/B'
            elif 35000 <= ingreso < 85000:
                return 'C+'
            elif 11600 <= ingreso < 35000:
                return 'C'
            elif 6800 <= ingreso < 11600:
                return 'D+'
            elif 2700 <= ingreso < 6800:
                return 'D'
            elif 0 <= ingreso < 2700:
                return 'E'
    return None

In [131]:
# Función para contar el total de operaciones por cuenta
def total_operaciones(cuenta_cliente):
  """Retorna el número total de operaciones por cuenta."""
  Portador = []
  for ii in cuenta_cliente:
    Sumita = 0
    for ij in range(len(lista_operaciones)):
      for k in range(len(Dat_fes)):
        try:
          if Dat_fes[k][mes] == lista_operaciones[ij]['MES']:
            if ii == lista_operaciones[ij]['CUENTA']:
               Sumita += 1
        except KeyError:
          pass
    Portador.append(Sumita)
  return Portador

In [132]:
# Función para calcular el saldo de cada cuenta
def Saldos(mes,cuenta_cliente):
  """Retorna el saldo de cada cuenta."""
  Saldo = []
  for ii in cuenta_cliente:
    S1 = 0
    for ij in range(len(lista_operaciones)):
      for k in range(len(Dat_fes)):
        try:
          if Dat_fes[k][mes] == lista_operaciones[ij]['MES']:
            if ii == lista_operaciones[ij]['CUENTA']:
              if lista_operaciones[ij]['TIPO OPERACIÓN'] == 1:
                S1 -= lista_operaciones[ij]['IMPORTE']
              else:
                S1 += lista_operaciones[ij]['IMPORTE']
        except KeyError:
          pass
    Saldo.append(round(S1,2))
  return Saldo

In [149]:
# Función para calcular el pago mínimo

# Función R (contador de meses)
def R(mes):
    tt = 10
    r = 0
    while tt <= mes:
        r += 1
        tt += 1
    return r

# Función para calcular el pago mínimo
def calcular_pago_minimo(mes, r, TATA,numero_cliente):
    inicio_del_mes = 0
    debito = 0  # Inicializar debito con un valor predeterminado
    for hh in range(r):
        for xx in range(len(TATA)):
            if TATA[xx] == 'TDC':
                Super_duplex = Saldos(mes - r + hh + 1, obtener_cuentas_cliente(numero_cliente))
                if Super_duplex[xx] <= 0:
                    pago_minimo = inicio_del_mes * 0.4 + Super_duplex[xx]
                else:
                    pago_minimo = 0
            else:
                debito = 0  # Actualizar debito si es una tarjeta de débito
                pago_minimo = 0
            Pago = round(np.abs(pago_minimo),2)
        inicio_del_mes = pago_minimo

    if len(obtener_cuentas_cliente(numero_cliente)) == 2:
        return [Pago, round(debito,2)]  # Retorna una lista
    else:
        if TATA[0] == 'TDC':
            return [Pago]  # Retorna una lista con un solo elemento
        else:
            return [round(debito,2)]  # Retorna una lista con un solo elemento

In [134]:
# Función para verificar el cumplimiento de pagos
def verificar_cumplimiento(cuentas, tipos_tarjeta, pagos_minimos,mes):
    """Verifica si se cumplió con el pago mínimo."""
    cumplimiento = []
    for cuenta, tipo, pago in zip(cuentas, tipos_tarjeta, pagos_minimos):
        if tipo == 'TDC':
            saldo = Saldos(mes,[cuenta])[0]
            cumplimiento.append('SI' if saldo >= pago else 'NO')
        else:
            cumplimiento.append('NA')  # No aplica para débito
    return cumplimiento

**NOTA 1:**

* UNA VEZ CARGADOS TODOS LOS CODIGOS DE ARRIBA, SOLO ES
NECESARIO VOLVER A CARGAR DESDE AQUÍ PARA LA EJECUCIÓN DE LOS REPORTES DE LAS FECHAS FALTANTES.

* CONSIDERA QUE AL INGRESAR EL MES SOLICITADO, TENDRÁS LA OPCIÓN DE INGRESAR 10 (CUANDO SOLICITES EL REPORTE DE OCTUBRE), 11 (CUANDO SOLICITES EL REPORTE DE NOVIEMBRE), 12 (CUANDO SOLICITES EL REPORTE DE DICIEMBRE).


In [146]:
# Solicitar mes y número de cliente
mes = int(input("Ingrese el mes (10, 11, 12): "))

#Valor arbitrario al cliente
numero_cliente=10

Ingrese el mes (10, 11, 12): 12


**NOTA 2:**

* SOLO EXISTEN 4 CLIENTES, POR LO CUAL, AL SOLICITARTE CUÁNTOS CLIENTES SON, DEBERÁS INGRESAR EL NÚMERO 4.

* LOS 4 TIPOS DE NÚMEROS DE CLIENTES QUE EXISTEN SON: 10, 20, 30 Y 40, DEBIDO A ESTO SE TE SOLICITARÁ 4 VECES QUE INGRESES EL NÚMERO DE CLIENTE, POR LO TANTO, DEBES DE INGRESAR PRIMERO "10", SEGUNDO "20", TERCERO "30" Y CUARTO "40".


In [147]:
20# Función principal para generar la matriz de resultados
def generar_matriz(clientes, mes):
    """Genera una matriz con los datos de los clientes y sus cuentas."""
    matriz = []
    for cliente in clientes:
        cuentas = obtener_cuentas_cliente(cliente)
        tip = Tarjetas_d_o_c(cuentas)
        tipos = Resulta(tip)
        edades = [calcular_edad(cliente)] * len(cuentas)
        niveles = [nivel_socioeconomico(cliente)] * len(cuentas)
        operaciones = total_operaciones(cuentas)
        saldos = Saldos(mes,cuentas)
        pagos_minimos = calcular_pago_minimo(mes, R(mes), Tarjetas_d_o_c(cuentas),cliente)
        cumplimiento = verificar_cumplimiento(cuentas, tip, pagos_minimos,mes)

        for i in range(len(cuentas)):
            fila = [
                str(fecha(mes)),  # Fecha
                cuentas[i],  # Número de cuenta
                tipos[i],  # Tipo de tarjeta
                edades[i],  # Edad
                niveles[i],  # Nivel socioeconómico
                operaciones[i],  # Total de operaciones
                saldos[i],  # Saldo
                pagos_minimos[i],  # Pago mínimo
                cumplimiento[i]  # Cumplimiento
            ]
            matriz.append(fila)
    return matriz

# Solicitar número de clientes y sus números
Numero_de_cliente = int(input('Digite cuántos clientes son: '))
Lista_final = [int(input('Digite número de cliente: ')) for _ in range(Numero_de_cliente)]

# Generar y mostrar la matriz
matriz = generar_matriz(Lista_final, mes)
for fila in matriz:
    print(fila)

Digite cuántos clientes son: 4
Digite número de cliente: 10
Digite número de cliente: 20
Digite número de cliente: 30
Digite número de cliente: 40
['2024-12-31', 234567, 100, 39, 'C+', 6, -4000.89, 0, 'NO']
['2024-12-31', 654321, 200, 39, 'C+', 0, 0, 0, 'NA']
['2024-12-31', 98765, 200, 39, 'D', 0, 0, 0, 'NA']
['2024-12-31', 1234, 100, 33, 'C', 6, -300.0, 5532.0, 'NO']
['2024-12-31', 543678, 100, 74, 'D+', 8, -630.0, 630.0, 'NO']


**UNA VEZ INGRESADO EL MES DEL REPORTE QUE DESEAS SOLICITAR, SOLO CARGAR EL REPORTE DEL MES QUE SOLICITASTE PARA PODER ADQUIRIR DICHO REPORTE**

Considera que volveras a ingresar la cantidad de clientes (4) y su respectivo número de cliente (10,20,30,40), como anteriormente lo ingresaste.

**REPORTE MES DE OCTUBRE**

In [142]:

# Función para guardar la matriz en un archivo de texto plano con datos centrados
def guardar_matriz_en_txt(matriz, nombre_archivo):
    """Guarda la matriz en un archivo de texto plano, centrando los valores en cada columna."""
    # Definir el ancho de cada columna
    anchos = [12, 10, 8, 6, 6, 8, 12, 12, 8]  # Ajusta estos valores según tus necesidades

    with open(nombre_archivo, 'w') as archivo:
        for fila in matriz:
            # Formatear cada valor para que esté centrado en su columna
            fila_formateada = []
            for i, valor in enumerate(fila):
                valor_str = str(valor)
                espacio_extra = anchos[i] - len(valor_str)
                izquierda = espacio_extra // 2
                derecha = espacio_extra - izquierda
                valor_centrado = ' ' * izquierda + valor_str + ' ' * derecha
                fila_formateada.append(valor_centrado)

            # Unir las columnas con '|' y escribir la fila en el archivo
            fila_str = '|'.join(fila_formateada)
            archivo.write(fila_str + '\n')

# Solicitar número de clientes y sus números
Numero_de_cliente = int(input('Digite cuántos clientes son: '))
Lista_final = [int(input('Digite número de cliente: ')) for _ in range(Numero_de_cliente)]

# Generar la matriz
matriz = generar_matriz(Lista_final, mes)

# Guardar la matriz en un archivo de texto plano
nombre_archivo = 'ESTATUS TARJETAS 202410.txt'
guardar_matriz_en_txt(matriz, nombre_archivo)

print(f"La matriz se ha guardado en el archivo: {nombre_archivo}")

Digite cuántos clientes son: 4
Digite número de cliente: 10
Digite número de cliente: 20
Digite número de cliente: 30
Digite número de cliente: 40
La matriz se ha guardado en el archivo: ESTATUS TARJETAS 202410.txt


**REPORTE MES DE NOVIEMBRE**

In [145]:
# Función para guardar la matriz en un archivo de texto plano con datos centrados
def guardar_matriz_en_txt(matriz, nombre_archivo):
    """Guarda la matriz en un archivo de texto plano, centrando los valores en cada columna."""
    # Definir el ancho de cada columna
    anchos = [12, 10, 8, 6, 6, 8, 12, 12, 8]  # Ajusta estos valores según tus necesidades

    with open(nombre_archivo, 'w') as archivo:
        for fila in matriz:
            # Formatear cada valor para que esté centrado en su columna
            fila_formateada = []
            for i, valor in enumerate(fila):
                valor_str = str(valor)
                espacio_extra = anchos[i] - len(valor_str)
                izquierda = espacio_extra // 2
                derecha = espacio_extra - izquierda
                valor_centrado = ' ' * izquierda + valor_str + ' ' * derecha
                fila_formateada.append(valor_centrado)

            # Unir las columnas con '|' y escribir la fila en el archivo
            fila_str = '|'.join(fila_formateada)
            archivo.write(fila_str + '\n')

# Solicitar número de clientes y sus números
Numero_de_cliente = int(input('Digite cuántos clientes son: '))
Lista_final = [int(input('Digite número de cliente: ')) for _ in range(Numero_de_cliente)]

# Generar la matriz
matriz = generar_matriz(Lista_final, mes)

# Guardar la matriz en un archivo de texto plano
nombre_archivo = 'ESTATUS TARJETAS 202411.txt'
guardar_matriz_en_txt(matriz, nombre_archivo)

print(f"La matriz se ha guardado en el archivo: {nombre_archivo}")

Digite cuántos clientes son: 4
Digite número de cliente: 10
Digite número de cliente: 20
Digite número de cliente: 30
Digite número de cliente: 40
La matriz se ha guardado en el archivo: ESTATUS TARJETAS 202411.txt


**REPORTE MES DE DICIEMBRE**

In [148]:
# Función para guardar la matriz en un archivo de texto plano con datos centrados
def guardar_matriz_en_txt(matriz, nombre_archivo):
    """Guarda la matriz en un archivo de texto plano, centrando los valores en cada columna."""
    # Definir el ancho de cada columna
    anchos = [12, 10, 8, 6, 6, 8, 12, 12, 8]  # Ajusta estos valores según tus necesidades

    with open(nombre_archivo, 'w') as archivo:
        for fila in matriz:
            # Formatear cada valor para que esté centrado en su columna
            fila_formateada = []
            for i, valor in enumerate(fila):
                valor_str = str(valor)
                espacio_extra = anchos[i] - len(valor_str)
                izquierda = espacio_extra // 2
                derecha = espacio_extra - izquierda
                valor_centrado = ' ' * izquierda + valor_str + ' ' * derecha
                fila_formateada.append(valor_centrado)

            # Unir las columnas con '|' y escribir la fila en el archivo
            fila_str = '|'.join(fila_formateada)
            archivo.write(fila_str + '\n')

# Solicitar número de clientes y sus números
Numero_de_cliente = int(input('Digite cuántos clientes son: '))
Lista_final = [int(input('Digite número de cliente: ')) for _ in range(Numero_de_cliente)]

# Generar la matriz
matriz = generar_matriz(Lista_final, mes)

# Guardar la matriz en un archivo de texto plano
nombre_archivo = 'ESTATUS TARJETAS 202412.txt'
guardar_matriz_en_txt(matriz, nombre_archivo)

print(f"La matriz se ha guardado en el archivo: {nombre_archivo}")

Digite cuántos clientes son: 4
Digite número de cliente: 10
Digite número de cliente: 20
Digite número de cliente: 30
Digite número de cliente: 40
La matriz se ha guardado en el archivo: ESTATUS TARJETAS 202412.txt


**Al finalizar el proceso, puedes adquirir otro reporte, volviendo a ejecutar nuevamente todo a partir de la Nota 1, ingresando la fecha del reporte que deseas solicitar**